In [35]:
import glob 
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim
from music21 import converter, instrument, note, chord
import time

In [36]:
def get_notes():
    
    notes = []
    
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        #print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
        
    return notes


In [37]:
notes = get_notes() 

pitchnames = sorted(set(item for item in notes))

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

vocab_size = len(note_to_int)

seq_len = 100

    
in_seq_0 = []
tar_seq_0 = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - seq_len, 1):
    sequence_in = notes[i:i + seq_len]
    sequence_out = notes[i + seq_len]
    
    for j in range(0, seq_len, 1):
        in_seq_0.append(note_to_int[sequence_in[j]])
        
    tar_seq_0.append(note_to_int[sequence_out])
# save the input- and outputsseq to the file 
with open('notes_in', 'wb') as fp:
    pickle.dump(in_seq_0, fp)
    
with open('notes_tar', 'wb') as fp:
    pickle.dump(tar_seq_0, fp)
    

In [38]:
class LSTM(nn.Module):
    
    def __init__(self, input_size, embedding_dim, batch_size, hidden_dim, output_size):
        super(LSTM, self).__init__()
        
        self.batch_size = batch_size
        
        self.hidden_dim = hidden_dim
        
        self.embeddings = nn.Embedding(input_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.linear = nn.Linear(hidden_dim, output_size)
        

    def init_hidden(self):

        return (torch.zeros(1, self.batch_size, self.hidden_dim).cuda(),
                torch.zeros(1, self.batch_size, self.hidden_dim).cuda())
    

    def forward(self, inputs):
        
        hidden = self.init_hidden()
        
        embeds = self.embeddings(inputs)
        
        lstm_out, hidden = self.lstm(embeds.view(len(net_in_tensor), 1, -1), hidden)
        
        prediction = self.linear(lstm_out.view(len(net_in_tensor), -1))
        pre_scores = F.log_softmax(prediction, dim=1)
        return pre_scores 

In [39]:
input_size = vocab_size

embedding_dim = 30
batch_size = 1
hidden_dim = 36
learning_rate = 0.01

model = LSTM(input_size, embedding_dim, batch_size, hidden_dim, vocab_size)
model.cuda()

model.zero_grad()
model.hidden = model.init_hidden()

lossfunction = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)


Epochs = 500

save_every = 50
print_every = 10

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

start = time.time()
total_Loss = []

for e in range(Epochs):
    
    total_Loss.append(0)
    counter = 0
    with open ('notes_in', 'rb') as fp:
        in_seq = pickle.load(fp)
    
    with open ('notes_tar', 'rb') as fp:
        tar_seq = pickle.load(fp)
    
    while len(in_seq) != 0:
        
        if (len(in_seq) >= 100 and len(tar_seq) != 0):
            counter += 1
            optimizer.zero_grad()
            
            net_in = in_seq[:100]
            del in_seq[:100]
            net_in_tensor= torch.tensor(net_in, dtype=torch.long).cuda()
            net_out = model(net_in_tensor)
            #get the last list of output array as output1
            net_out1 = torch.narrow(net_out, 0, 99, 1)
            target1 = torch.tensor([tar_seq.pop()], dtype=torch.long).cuda()
            
            if (len(in_seq) >= 100 and len(tar_seq) != 0):
                counter += 1
                optimizer.zero_grad()
                net_in = in_seq[:100]
                del in_seq[:100]
                net_in_tensor= torch.tensor(net_in, dtype=torch.long).cuda()
                net_out = model(net_in_tensor)
                #get the last list of output array as output2
                net_out2 = torch.narrow(net_out, 0, 99, 1)
                target2 = torch.tensor([tar_seq.pop()], dtype=torch.long).cuda()
                #change the dimention of the inputs of lossfunction from 1 to 2
                output_seq = torch.cat((net_out1, net_out2), 0)
                target_seq = torch.cat((target1, target2), 0)
                loss = lossfunction(output_seq, target_seq)
            
                loss.backward()
                optimizer.step()

                total_Loss[-1] += loss 
            else:
                break         
        else:
            break
    #save the weights of the Model        
    if e % save_every == 0:
        torch.save(model.state_dict(), f'./net_{e}.pth')
    #print the Loss and Time of the training 
    if e % print_every == 0:   
        print('Epoch {}: Total Loss = {}, Avg. Time/Epoch = {}'
                .format(e, total_Loss[-1], (time.time() - start) / print_every))
        start = time.time()

cuda:0
Epoch 0: Total Loss = 14191.2861328125, Avg. Time/Epoch = 8.89884386062622
Epoch 10: Total Loss = 13174.861328125, Avg. Time/Epoch = 86.58851318359375
Epoch 20: Total Loss = 12990.9951171875, Avg. Time/Epoch = 86.9698876619339
Epoch 30: Total Loss = 13117.634765625, Avg. Time/Epoch = 82.17859997749329
Epoch 40: Total Loss = 12895.453125, Avg. Time/Epoch = 83.4453234910965
Epoch 50: Total Loss = 12946.40234375, Avg. Time/Epoch = 86.04122703075409
Epoch 60: Total Loss = 12998.224609375, Avg. Time/Epoch = 81.96825749874115
Epoch 70: Total Loss = 12941.822265625, Avg. Time/Epoch = 91.37112436294555
Epoch 80: Total Loss = 13002.9228515625, Avg. Time/Epoch = 90.20939438343048
Epoch 90: Total Loss = 12928.4443359375, Avg. Time/Epoch = 86.07905864715576
Epoch 100: Total Loss = 13099.7568359375, Avg. Time/Epoch = 81.65858454704285
Epoch 110: Total Loss = 12998.28125, Avg. Time/Epoch = 87.99343454837799
Epoch 120: Total Loss = 13029.65234375, Avg. Time/Epoch = 81.33473708629609
Epoch 130:

KeyboardInterrupt: 

In [ ]:
print( net_out1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(total_Loss)